In [1]:
import pandas as pd
import numpy as np
import glob, os
import xlrd
from collections import defaultdict

In [2]:
#os.chdir("/home/sara/Documents/Immigration/Shared_statistics/")
resident_foreigners_norm = pd.read_table("/home/sara/Documents/Immigration/Shared_statistics/Data_final/resident_foreigners_norm.csv")

province_capitals = sorted(list(set(resident_foreigners_norm["Province"])))

In [3]:
prov_codes = pd.read_table("/home/sara/Documents/Immigration/Data_not_git/Data_distances/Elenco-comuni-italiani.csv", sep= ";", 
                           usecols = ['Codice Regione', 'Codice Città Metropolitana', 
                                      'Codice Comune numerico con 110 province (dal 2010 al 2016)', 
                                      'Denominazione corrente',
                                      'Denominazione regione'])

villasangiovanni_code = prov_codes[(prov_codes["Denominazione corrente"] == "Villa San Giovanni") & (prov_codes["Denominazione regione"] == "Calabria")]["Codice Comune numerico con 110 province (dal 2010 al 2016)"].values[0]
civitavecchia_code = prov_codes[(prov_codes["Denominazione corrente"] == "Civitavecchia") & (prov_codes["Denominazione regione"] == "Lazio")]["Codice Comune numerico con 110 province (dal 2010 al 2016)"].values[0]

# Some territories approximations
prov_codes['Denominazione corrente'].replace(['Verbania', 'Monza', 'Forlì', 'Massa', 'Pesaro', 'Aosta', 'Bolzano', 'Sanluri', 'Olbia', 'Barletta', 'Carbonia', 'Tortolì'], ['Verbano-Cusio-Ossola', 'Monza e della Brianza', 'Forlì-Cesena', 'Massa-Carrara', 'Pesaro e Urbino', "Valle d'Aosta / Vallée d'Aoste", 'Bolzano / Bozen', 'Medio Campidano', 'Olbia-Tempio', 'Barletta-Andria-Trani', 'Carbonia-Iglesias', 'Ogliastra'], inplace = True)

In [4]:
prov_codes.head()

,Codice Regione,Codice Città Metropolitana,Denominazione corrente,Denominazione regione,Codice Comune numerico con 110 province (dal 2010 al 2016)
0,1.0,201,Agliè,Piemonte,1001.0
1,1.0,201,Airasca,Piemonte,1002.0
2,1.0,201,Ala di Stura,Piemonte,1003.0
3,1.0,201,Albiano d'Ivrea,Piemonte,1004.0
4,1.0,201,Alice Superiore,Piemonte,1005.0


In [5]:
prov_codes = prov_codes[prov_codes["Denominazione corrente"].isin(province_capitals)]
prov_codes.index = range(len(prov_codes))

In [6]:
prov_codes.head()

,Codice Regione,Codice Città Metropolitana,Denominazione corrente,Denominazione regione,Codice Comune numerico con 110 province (dal 2010 al 2016)
0,1.0,201,Torino,Piemonte,1272.0
1,1.0,-,Vercelli,Piemonte,2158.0
2,1.0,-,Novara,Piemonte,3106.0
3,1.0,-,Cuneo,Piemonte,4078.0
4,1.0,-,Asti,Piemonte,5005.0


In [7]:
prov_codes.loc[prov_codes["Denominazione corrente"] == "Bolzano / Bozen", "Denominazione regione"] = "Bolzano / Bozen"

In [8]:
prov_codes.loc[prov_codes["Denominazione corrente"] == "Trento", "Denominazione regione"] = "Trento"

In [9]:
prov_codes = prov_codes.replace({"Valle d'Aosta/Vallée d'Aoste": "Valle d'Aosta / Vallée d'Aoste"})

In [10]:
prov_codes.to_csv("/home/sara/Documents/Immigration/Data_not_git/Data_distances/Elenco-comuni-italiani_update.csv", sep = "\t")

In [11]:
all_codes = prov_codes["Codice Comune numerico con 110 province (dal 2010 al 2016)"].values

In [12]:
name_codes_corr = prov_codes.set_index('Codice Comune numerico con 110 province (dal 2010 al 2016)')['Denominazione corrente'].to_dict()

## About the ISTAT data:

Data Source: http://www.istat.it/it/archivio/157423

ISTAT releases the origin-destination matrices of distances in meters and travel times (in minutes) between all Italian municipalities. The matrices are grouped by Region. The files are provided in text format and all Italian municipalities. 
The islands are treated separately. The matrices of Sicily and Sardinia contain only the between-region distances (the distances only of the municipalities of the regions). 
In a separate excel file are available the distances for the main ports that connect the islands with respect to Peninsular Italy. This allows you to add the travel time by ship to the routes calculated from the main ports of connection to the islands.
 
Once again there are some coherence problems within the different tables:
- the column names are not always the same;
- column are index may be inverted: for all the regions but Lombardia the region-provinces are in the "Destination" column and the other provinces in the "Origin" column. In the Lombardia table the two data are inverted (thus, the need to transpose the matrix).

Morover I am not able to find the Sicilia and Sardinia ports distances!

In [47]:
# Initialize an empty DataFrame for the distances-matrix
distances = pd.DataFrame()


os.chdir("/home/sara/Documents/Immigration/Data_not_git/Data_distances/Regions")
for file in glob.glob("*.txt"):
    print(file)
    try:
        temp = pd.read_table(file, sep = ";", usecols = ["Origine", "Destinazione", "Total_Mete"])
        temp.rename(columns={'Origine': 'Origin', 'Destinazione': 'Destination', 'Total_Mete': 'Distance'}, inplace=True)
    except ValueError:
        temp = pd.read_table(file, sep = ";", usecols = ["origine", "Destinazione", "Total_Mete"])
        temp.rename(columns={'origine': 'Origin', 'Destinazione': 'Destination', 'Total_Mete': 'Distance'}, inplace=True)

    for c in temp.columns:
        temp[c] = temp[c].str.replace(",", ".")
        temp[c] = temp[c].astype(float)
        
    if file.split(".")[0] == "Calabria":
        temp_villasangiovanni = temp[(temp["Destination"] == villasangiovanni_code) & (temp["Origin"].isin(all_codes))]
        temp_villasangiovanni = temp_villasangiovanni.replace(name_codes_corr)
        villasangiovanni = temp_villasangiovanni.set_index('Origin')['Distance'].to_dict()
        
    if file.split(".")[0] == "Lazio":
        temp_civitavecchia = temp[(temp["Destination"] == civitavecchia_code) & (temp["Origin"].isin(all_codes))]
        temp_civitavecchia = temp_civitavecchia.replace(name_codes_corr)
        civitavecchia = temp_civitavecchia.set_index('Origin')['Distance'].to_dict()
        
    temp = temp[(temp["Origin"].isin(all_codes)) & (temp["Destination"].isin(all_codes))]
    temp = temp.replace(name_codes_corr)

    temp = temp.pivot(index='Origin', columns='Destination', values='Distance')
    
    if file.split(".")[0] == "Lombardia":
        temp = temp.transpose()
    
    distances = pd.concat([distances, temp], axis=1)

Abruzzo.txt
Basilicata.txt
Calabria.txt
Campania.txt
Emilia.txt
Friuli.txt
Lazio.txt
Liguria.txt
Lombardia.txt
Marche.txt
Molise.txt
Piemonte.txt
Puglia.txt
Sardegna.txt
Sicilia.txt
Toscana.txt
Trentino.txt
Umbria.txt
ValledAosta.txt
Veneto.txt


In [48]:
distances.head()

,Chieti,L'Aquila,Pescara,Teramo,Matera,Potenza,Catanzaro,Cosenza,Crotone,Reggio di Calabria,...,Perugia,Terni,Valle d'Aosta / Vallée d'Aoste,Belluno,Padova,Rovigo,Treviso,Venezia,Verona,Vicenza
Agrigento,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alessandria,617577.83,639953.48,606831.64,586073.06,961908.27,922857.68,1170784.15,1079238.41,1158004.65,1263295.05,...,461149.98,538617.32,167420.83,433768.99,313411.48,320741.23,365648.24,345261.68,235727.45,284071.30
Ancona,166344.52,188720.17,155598.33,134839.76,510674.97,451533.00,804909.01,713363.27,753470.22,897419.92,...,158272.08,183651.48,612167.83,450624.64,325491.73,289541.48,382503.90,362117.34,357362.77,360097.99
Arezzo,380979.42,302482.56,392945.04,359200.33,646521.63,550259.47,798185.94,706640.20,785406.44,890696.85,...,73317.84,166019.12,551609.38,420146.75,295013.84,259063.59,352026.00,331639.44,307940.44,329620.10
Ascoli Piceno,101160.74,93242.70,90414.56,34923.05,445491.19,386349.22,739725.23,648179.49,688286.44,832236.14,...,218542.57,141347.16,721815.27,560272.08,435139.17,399188.92,492151.34,471764.78,467010.21,469745.43


To compute the distances between Sicily-Sardinia and the othe Italian regions, it is possible to manually detect (GoogleMaps) some useful distances between the main ports and compute the distances as the sum of multiple component. 

In particular:

from Sicily:
- $d_{(origin province, Trapani)} + d_{(Trapani, Cagliari)} + d_{(Cagliari, destination province)}$ if destination province in Sardinia;
- $d_{(origin province, Messina)} + d_{(Messina, Villa San Giovanni)} + d_{(Villa San Giovanni, destination province)}$ otherwise;

from Sardinia:
- $d_{(origin province, Cagliari)} + d_{(Cagliari, Trapani)} + d_{(Trapani, destination province)}$ if destination province in Sicily;
- $min(d_{(origin province, Cagliari)} + d_{(Cagliari, Trapani)} + d_{(Trapani, Messina)} + d_{(Messina, Villa San Giovanni)} + d_{(Villa San Giovanni, destination province)}, d_{(origin province, Olbia)} + d_{(Olbia, Livorno)} + d_{(Livorno, destination province)}, d_{(origin province, Olbia)} + d_{(Olbia, Civitavecchia)} + d_{(Civitavecchia, destination province)})$ otherwise;

In [49]:
useful = {"Messina-VillaSanGiovanni": 10300, "Cagliari-Trapani": 361000, "Olbia-Civitavecchia": 237000, 
         "Olbia-Livorno": 313000}

In [50]:
distances_approx = distances.copy()
for c in distances_approx.columns:
    index = distances_approx.index[distances_approx[c].isnull()]
    for i in index:
        # Sicily - Sardinia distances
        if (prov_codes[prov_codes["Denominazione corrente"] == i]["Denominazione regione"].values[0] == "Sicilia" and prov_codes[prov_codes["Denominazione corrente"] == c]["Denominazione regione"].values[0] == "Sardegna"):
            temp = distances[i]["Trapani"] + useful["Cagliari-Trapani"] + distances["Cagliari"][c]

        '''if prov_codes[prov_codes["Denominazione corrente"] == c]["Denominazione regione"].values[0] == "Sicilia" and prov_codes[prov_codes["Denominazione corrente"] == i]["Denominazione regione"].values[0] == "Sardegna":
            print(i, c)
            print(distances[c]["Trapani"] + useful["Cagliari-Trapani"] + distances["Cagliari"][i])'''

        # Sicily - Peninsular Italy
        if (prov_codes[prov_codes["Denominazione corrente"] == i]["Denominazione regione"].values[0] == "Sicilia" and prov_codes[prov_codes["Denominazione corrente"] == c]["Denominazione regione"].values[0] != "Sardegna"):
            temp = distances[i]["Messina"] + useful["Messina-VillaSanGiovanni"] + villasangiovanni[c]

        # Sardinia - Peninsular Italy    
        if (prov_codes[prov_codes["Denominazione corrente"] == i]["Denominazione regione"].values[0] == "Sardegna" and prov_codes[prov_codes["Denominazione corrente"] == c]["Denominazione regione"].values[0] != "Sicilia"):
            temp = min(distances[i]["Olbia-Tempio"] + useful["Olbia-Civitavecchia"] + civitavecchia[c], 
                       distances[i]["Olbia-Tempio"] + useful["Olbia-Livorno"] + distances["Livorno"][c],
                       distances[i]["Cagliari"] + useful["Cagliari-Trapani"] + distances["Trapani"]["Messina"] + useful["Messina-VillaSanGiovanni"] + villasangiovanni[c])
        distances_approx[c][i] = temp
        distances_approx[i][c] = temp

The distance matrix is not squared. This may due to measures error. Let's force it to be squared.

In [107]:
distances_approx = distances_approx.reindex(sorted(distances_approx.columns), axis=1)

for i in distances_approx.index:
    for j in distances_approx.columns.tolist()[distances_approx.columns.tolist().index(i):]:
        if (distances_approx[i][j] != distances_approx[j][i]):
            distances_approx[j][i] = distances_approx[i][j]

In [110]:
W = distances_approx.reindex(sorted(distances_approx.columns), axis=1)
W.to_csv("/home/sara/Documents/Immigration/Shared_models/Data/Distances_matrix.csv", sep = "\t")

In [111]:
W.head()

,Agrigento,Alessandria,Ancona,Arezzo,Ascoli Piceno,Asti,Avellino,Bari,Barletta-Andria-Trani,Belluno,...,Udine,Valle d'Aosta / Vallée d'Aoste,Varese,Venezia,Verbano-Cusio-Ossola,Vercelli,Verona,Vibo Valentia,Vicenza,Viterbo
Agrigento,0.00,1.520282e+06,1.154407e+06,572277.040000,1.089223e+06,1.552215e+06,731677.380000,703529.240000,754750.580000,572277.04000,...,1.576328e+06,572277.040000,1.569916e+06,572277.040000,1.597015e+06,1.569912e+06,572277.04000,3.619691e+05,572277.040000,1.037766e+06
Alessandria,1520281.89,0.000000e+00,4.573027e+05,385366.497500,5.681687e+05,3.763389e+04,810608.562500,913715.338125,863973.061875,434130.11125,...,4.573015e+05,167729.035625,1.419872e+05,346190.763750,1.418589e+05,5.552813e+04,235796.18125,1.173750e+06,285590.234375,5.228014e+05
Ancona,1154406.75,4.573027e+05,0.000000e+00,242641.575625,1.141482e+05,4.907148e+05,438618.041875,456566.656875,404576.023125,447901.06250,...,4.734930e+05,608873.778750,4.847525e+05,359845.529375,5.120943e+05,5.040044e+05,354842.31500,8.105737e+05,356127.051875,2.474618e+05
Arezzo,572277.04,3.853665e+05,2.426416e+05,0.000000,2.858283e+05,4.210109e+05,436974.663125,631590.724375,579600.096250,420049.72125,...,4.456417e+05,550710.037500,4.368461e+05,331994.182500,4.651546e+05,4.387711e+05,307902.62250,7.997079e+05,328275.710625,1.477071e+05
Ascoli Piceno,1089222.97,5.681687e+05,1.141482e+05,285828.296250,0.000000e+00,6.013101e+05,373114.746875,389874.518125,337883.884375,558496.29375,...,5.840883e+05,719469.010000,5.937786e+05,470440.760625,6.226896e+05,6.145996e+05,465437.54625,7.450704e+05,466722.283125,2.327132e+05


### Region Distance Matrix

It could be useful to use different methods to build the distance matrix between regions (zones):
- the distance between the two regions capitals
- the mean distance between all the province pairs belonging to the two regions (zones) considered

In [112]:
regions_capitals = {'Ancona': 'Marche', 'Bari': 'Puglia', 'Bologna': 'Emilia-Romagna', 
                    'Bolzano / Bozen': 'Bolzano / Bozen', 'Cagliari': 'Sardegna', 'Campobasso': 'Molise',
                    'Catanzaro': 'Calabria', 'Firenze': 'Toscana', 'Genova': 'Liguria', "L'Aquila": 'Abruzzo',
                    'Milano': 'Lombardia', 'Napoli': 'Campania', 'Palermo': 'Sicilia', 'Perugia': 'Umbria',
                    'Potenza': 'Basilicata', 'Roma': 'Lazio', 'Torino': 'Piemonte', 'Trento': 'Trento',
                    'Trieste': 'Friuli-Venezia Giulia', 
                    "Valle d'Aosta / Vallée d'Aoste": "Valle d'Aosta / Vallée d'Aoste", 'Venezia': 'Veneto'}

In [113]:
W_regions_capitals = W.loc[sorted(list(regions_capitals.keys()))][sorted(list(regions_capitals.keys()))]
W_regions_capitals.columns = [regions_capitals[k] for k in W_regions_capitals.columns]
W_regions_capitals.index = [regions_capitals[k] for k in W_regions_capitals.index]

In [114]:
W_regions_capitals.to_csv("/home/sara/Documents/Immigration/Shared_models/Data/Region_distances_matrix_regions_capitals.csv", sep = "\t")

In [115]:
W_regions_capitals.head()

,Marche,Puglia,Emilia-Romagna,Bolzano / Bozen,Sardegna,Molise,Calabria,Toscana,Liguria,Abruzzo,...,Campania,Sicilia,Umbria,Basilicata,Lazio,Piemonte,Trento,Friuli-Venezia Giulia,Valle d'Aosta / Vallée d'Aoste,Veneto
Marche,0.000000,4.565667e+05,208806.974375,491219.236875,883813.78,315759.780000,8.081049e+05,283839.193125,510585.102500,188242.691250,...,421730.727500,1127490.54,158696.153125,468024.759375,293445.163750,543947.876250,435101.805625,5.080633e+05,6.088738e+05,359845.529375
Puglia,456566.656875,0.000000e+00,666943.408125,946900.403125,1021684.47,218546.333125,3.524077e+05,699768.263750,935855.238750,395213.814375,...,261858.605625,676613.03,582696.140000,126306.198125,443370.643125,999629.042500,890782.971875,9.637444e+05,1.064555e+06,815526.695625
Emilia-Romagna,208806.974375,6.669434e+05,0.000000,280333.649375,757339.65,526318.750625,9.642384e+05,103590.788750,299460.946250,395539.392500,...,577677.384375,1287748.26,249085.630000,711786.074375,384796.012500,333062.288750,224216.213750,2.994882e+05,3.979882e+05,151434.352500
Bolzano / Bozen,491219.236875,9.469004e+05,280333.649375,0.000000,534728.97,805545.361875,1.226815e+06,366256.013125,406671.080625,674303.096250,...,840854.458125,572277.04,517731.550000,978712.155625,649277.632500,411475.818125,58634.423750,4.123151e+05,4.540391e+05,266567.426250
Sardegna,883813.780000,1.021684e+06,757339.650000,534728.970000,0.00,795154.570000,8.501003e+05,601428.050000,757850.370000,698982.040000,...,793606.820000,462767.15,462767.150000,930456.270000,579900.750000,920858.650000,361000.000000,1.054722e+06,3.610000e+05,361000.000000


In [116]:
W_mean = pd.DataFrame(columns=W_regions_capitals.columns, index=W_regions_capitals.index)

In [117]:
grouped = prov_codes.groupby(["Denominazione regione"])["Denominazione corrente"]
for r in W_mean.columns:
    i_list = grouped.get_group(r).values
    for s in W_mean.columns[W_mean.loc[r].isnull()]:
        if r == s:
            W_mean.loc[r][s] = 0
            W_mean.loc[s][r] = 0
        else:
            j_list = grouped.get_group(s).values
            temp = []
            for i_single in i_list:
                for j_single in j_list:
                    temp.append(W.loc[i_single][j_single])
            temp = sum(temp) / len(temp)
            W_mean.loc[r][s] = temp
            W_mean.loc[s][r] = temp

In [118]:
W_mean.head()

,Marche,Puglia,Emilia-Romagna,Bolzano / Bozen,Sardegna,Molise,Calabria,Toscana,Liguria,Abruzzo,...,Campania,Sicilia,Umbria,Basilicata,Lazio,Piemonte,Trento,Friuli-Venezia Giulia,Valle d'Aosta / Vallée d'Aoste,Veneto
Marche,0,470851,259783,520896,757108,287912,786108,331154,541346,143189,...,413138,1.0707e+06,171033,473579,267751,540443,464616,502723,638388,387615
Puglia,470851,0,714849,976977,958868,273244,368562,746161,979401,373729,...,262875,661636,579798,144141,478157,996704,921101,959034,1.09487e+06,843996
Emilia-Romagna,259783,714849,0,295583,705246,530982,985199,204481,305790,384954,...,610935,1.26563e+06,304207,725183,416957,296082,241560,315611,394630,192708
Bolzano / Bozen,520896,976977,295583,0,658519,795016,1.22506e+06,419895,434998,646909,...,850531,572277,556608,987355,654779,376799,58634.4,355704,454039,206357
Sardegna,757108,958868,705246,658519,0,672280,917878,683039,674069,655718,...,703543,713416,718150,879530,516149,797887,484790,922474,484790,599713


In [119]:
W_mean.to_csv("/home/sara/Documents/Immigration/Shared_models/Data/Region_distances_matrix_mean.csv", sep = "\t")